In [1]:
import skforecast
import pandas as pd

In [4]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(5)

,GAME_ORDER,YEAR,DATE,GAME_NUMBER,WEEK,PLAYER,AGE,TEAM,OPPONENT,RESULT,...,PASSING_ATT,PASSING_CMP_PCT,PASSING_YDS,PASSING_TD,PASSING_INT,PASSING_RATE,PASSING_SK,PASSING_YDS1,PASSING_Y_A,PASSING_AY_A
0,0.0,2018.0,2018-09-09,1.0,1.0,Christian McCaffrey,22.094,CAR,DAL,W 16-8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,2018.0,2018-09-16,2.0,2.0,Christian McCaffrey,22.101,CAR,ATL,L 24-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,2018.0,2018-09-23,3.0,3.0,Christian McCaffrey,22.108,CAR,CIN,W 31-21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,2018.0,2018-10-07,4.0,5.0,Christian McCaffrey,22.122,CAR,NYG,W 33-31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,2018.0,2018-10-14,5.0,6.0,Christian McCaffrey,22.129,CAR,WAS,L 17-23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
